In [72]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets


In [73]:
IMAGE_SIZE = 32
Z_DIM = 128
EPOCH = 5

In [74]:
(x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()

In [75]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [76]:
def preprocess(imgs):
    imgs = imgs.astype("float32")/255
    imgs = np.pad(imgs,((0,0),(2,2),(2,2)),constant_values=0.0)
    imgs = np.expand_dims(imgs,-1)
    return imgs
    
train = preprocess(x_train)
test = preprocess(x_test)

print(train.shape)
print(train.shape)

(60000, 32, 32, 1)
(60000, 32, 32, 1)


In [77]:
from tensorflow.keras import layers,models,losses,metrics
import tensorflow.keras.backend as K

# Discriminator

In [78]:
discrminator_input = layers.Input(shape=(IMAGE_SIZE,IMAGE_SIZE,1),name="discriminator_input")

x = layers.Conv2D(64,kernel_size=(3,3),strides=2,padding="same")(discrminator_input)
x = layers.LeakyReLU()(x)
x = layers.Dropout(rate=0.2)(x)

x = layers.Conv2D(128,kernel_size=(3,3),strides=2,padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(rate=0.2)(x)

x = layers.Conv2D(256,kernel_size=(3,3),strides=2,padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(rate=0.2)(x)

x = layers.Conv2D(1,kernel_size=(4,4),strides=1,padding="valid",use_bias=False,activation="sigmoid")(x)
discrminator_output = layers.Flatten(name="discrminator_output")(x)

discrminator = models.Model(discrminator_input,discrminator_output)
discrminator.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 discriminator_input (Input  [(None, 32, 32, 1)]       0         
 Layer)                                                          
                                                                 
 conv2d_67 (Conv2D)          (None, 16, 16, 64)        640       
                                                                 
 leaky_re_lu_76 (LeakyReLU)  (None, 16, 16, 64)        0         
                                                                 
 dropout_56 (Dropout)        (None, 16, 16, 64)        0         
                                                                 
 conv2d_68 (Conv2D)          (None, 8, 8, 128)         73856     
                                                                 
 batch_normalization_62 (Ba  (None, 8, 8, 128)         512       
 tchNormalization)                                        

# Generator 

In [79]:
generator_input = layers.Input(shape=(Z_DIM,))

x = layers.Dense(2*2*16)(generator_input)
x = layers.Reshape((2,2,16))(x)

x = layers.Conv2DTranspose(32,kernel_size=(3,3),strides=2,padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(64,kernel_size=(3,3),strides=2,padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(128,kernel_size=(3,3),strides=2,padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256,kernel_size=(3,3),strides=2,padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

generator_output = layers.Conv2D(1,kernel_size=3,strides=1,padding="same",activation="sigmoid")(x)

generator = models.Model(generator_input,generator_output)
generator.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 reshape_5 (Reshape)         (None, 2, 2, 16)          0         
                                                                 
 conv2d_transpose_20 (Conv2  (None, 4, 4, 32)          4640      
 DTranspose)                                                     
                                                                 
 batch_normalization_64 (Ba  (None, 4, 4, 32)          128       
 tchNormalization)                                               
                                                                 
 leaky_re_lu_79 (LeakyReLU)  (None, 4, 4, 32)          0  

# GAN